In [ ]:
!wget -O RAG.zip https://github.com/mthanhlm/LLM_email_phishing/raw/refs/heads/main/data/RAG.zip
!unzip -o -q RAG.zip -d rag_dataset

--2024-11-30 17:11:51--  https://github.com/phuongkhanh201/phishing_dataset/raw/refs/heads/main/RAG.zip
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/phuongkhanh201/phishing_dataset/refs/heads/main/RAG.zip [following]
--2024-11-30 17:11:52--  https://raw.githubusercontent.com/phuongkhanh201/phishing_dataset/refs/heads/main/RAG.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5705532 (5.4M) [application/zip]
Saving to: ‘RAG.zip’

RAG.zip             100%[===================>]   5.44M  --.-KB/s    in 0.07s   

2024-11-30 17:11:52 (73.4 MB/s) - ‘RAG.zip’ saved [5705532/5705532]



In [ ]:
!apt-get install -y pciutils lshw
!lspci | grep -i nvidia

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libpci3 pci.ids usb.ids
The following NEW packages will be installed:
  libpci3 lshw pci.ids pciutils usb.ids
0 upgraded, 5 newly installed, 0 to remove and 49 not upgraded.
Need to get 883 kB of archives.
After this operation, 3,256 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 pci.ids all 0.0~2022.01.22-1 [251 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 libpci3 amd64 1:3.7.0-6 [28.9 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/main amd64 lshw amd64 02.19.git.2021.06.19.996aaad9c7-2build1 [321 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy/main amd64 pciutils amd64 1:3.7.0-6 [63.6 kB]
Get:5 http://archive.ubuntu.com/ubuntu jammy/main amd64 usb.ids all 2022.04.02-1 [219 kB]
Fetched 883 kB in 2s (586 kB/s)
Selecting previously unselected package pci.ids.
(Reading d

In [ ]:
!curl -fsSL https://ollama.com/install.sh | sh

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
############################################################################################# 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> NVIDIA GPU installed.
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [ ]:
import subprocess
import time

# Start the Ollama server
subprocess.Popen(["ollama", "serve"])

# Wait for a few seconds for Ollama to load
time.sleep(3)

In [ ]:

# Run the ollama pull command
!ollama pull gemma:2b
!ollama pull hf.co/daophuongkhanh201/gemma-2b:Q4_K_M


pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠧ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠋ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠸ pulling manifest ⠸ pulling manifest ⠼ pulling manifest 
pulling c1864a5eb193...   0% ▕▏    0 B/1.7 GB                  pulling manifest 
pulling c1864a5eb193...   0% ▕▏    0 B/1.7 GB                  pulling manifest 
pulling c1864a5eb193...   0% ▕▏    0 B/1.7 GB                  pulling manifest 
pulling c1864a5eb193...   1% ▕▏  10 MB/1.7 GB                  pulling manifest 
pulling c1864a5eb193...   3% ▕▏  45 MB/1.7 GB                  pulling manifest 
pulling c1864a5eb193...   4% ▕▏  74 MB/1.7 GB                  pulling manifest 
pulling c1864a5eb193...   6% ▕▏  97 MB/1.7 GB                  pulling manifest 
pulling c1864a5eb193...   7% ▕▏ 122 MB/1.7 GB                  pulling manifest 
pulling c1864a5eb193...   8% ▕▏ 129 MB/1.7 GB    

In [ ]:
%pip install sentence-transformers instructor openai tqdm scikit-learn -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.1/70.1 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.2/325.2 kB 12.7 MB/s eta 0:00:00


In [ ]:
import os
import random
import numpy as np
from pathlib import Path
from tqdm import tqdm
from typing import Optional
from pydantic import BaseModel
from sklearn.cluster import MiniBatchKMeans
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
from openai import OpenAI
import instructor
import numpy as np


# Định nghĩa model kết quả phân tích phishing đơn giản
class SimplePhishingResult(BaseModel):
    is_potential_phishing: bool  # Biến boolean cho biết email có phải là phishing hay không ()



class PhishingDetector:
    """
    Lớp PhishingDetector dùng để phát hiện email lừa đảo (phishing) sử dụng kết hợp:
    - Language Model (LLM)
    - RAG (Retrieval Augmented Generation)
    - Few-shot learning

    Attributes:
        MAX_NSHOT: Số lượng ví dụ tối đa cho few-shot learning
        MAX_CLUSTERS: Số lượng cụm tối đa cho việc tổ chức dữ liệu RAG
    """

    MAX_NSHOT = 7  # Giới hạn số lượng ví dụ cho few-shot learning
    MAX_CLUSTERS = 20  # Giới hạn số lượng cụm cho clustering

    def __init__(self, ollama_model_name="gemma:2b", rag_dataset_path=None, embedding_model="all-MiniLM-L6-v2",
                 use_rag=True, use_few_shot=True, nshot=7, top_k_rag=5, similarity_threshold=0.7):
        """
        Khởi tạo detector với các tham số:

        Args:
            ollama_model_name: Tên model LLM sử dụng
            rag_dataset_path: Đường dẫn đến dataset RAG
            embedding_model: Model để tạo embedding cho văn bản
            use_rag: Có sử dụng RAG hay không
            use_few_shot: Có sử dụng few-shot learning hay không
            nshot: Số lượng ví dụ few-shot
            top_k_rag: Số lượng context tương tự nhất lấy từ RAG
            similarity_threshold: Ngưỡng độ tương đồng cho việc lọc context
        """

        # Lưu các tham số khởi tạo
        self.ollama_model_name = ollama_model_name
        self.rag_dataset_path = Path(rag_dataset_path) if rag_dataset_path else None
        self.use_rag = use_rag
        self.use_few_shot = use_few_shot
        self.nshot = min(nshot, self.MAX_NSHOT)
        self.top_k_rag = top_k_rag
        self.similarity_threshold = similarity_threshold

        # Khởi tạo OpenAI client với Ollama endpoint
        self.client = instructor.from_openai(
            OpenAI(base_url="http://127.0.0.1:11434/v1", api_key="ollama"),
            mode=instructor.Mode.JSON
        )

        # Khởi tạo embedding model nếu sử dụng RAG
        self.embedding_model = SentenceTransformer(embedding_model) if use_rag else None  # biến đổi text -> vector
        self.kmeans = None
        self.cluster_centroids = None

        # Chuẩn bị ví dụ cho few-shot và dataset RAG
        self.few_shot_examples = self._prepare_few_shot_examples() if use_few_shot else []
        self.rag_dataset = self._load_rag_dataset() if use_rag and self.rag_dataset_path and self.rag_dataset_path.exists() else []

        # Tổ chức dataset RAG thành các cụm nếu có
        if self.rag_dataset:
            self._organize_rag_dataset()


    def _prepare_few_shot_examples(self):
        """
        Chuẩn bị các ví dụ cho few-shot learning, bao gồm:
        - Các email phishing mẫu
        - Các email hợp lệ mẫu

        Returns:
            List các ví dụ few-shot có định dạng {role, email, label}
        """
        phishing_examples = [
            """From: security@amazonsupport.net
To: john.doe@company.com
Subject: Urgent: Your Amazon Account Has Been Locked

Dear Valued Customer,

We have detected unusual activity on your Amazon account. To prevent unauthorized access, we have temporarily locked your account for your protection.

To unlock your account and restore full access, please click on the link below and verify your information:

https://amaz0n-account-verify.com/unlock-account

Thank you for your prompt attention.

Best regards,
Amazon Security Team""",
            """From: it-support@company.com
To: michael.jones@company.com
Subject: Immediate Action Required: Password Expiry Notice

Dear Michael,

Our records indicate that your password is set to expire within 24 hours. Please update your password immediately by clicking the secure link:

http://company-support-authenticate.com/reset-password

Best regards,
IT Support Team""",
            """From: rewards@customerservice.net
To: susan.miller@company.com
Subject: You’ve Earned a $500 Reward! Claim Now

Dear Susan,

Congratulations! You've been selected to receive a $500 reward. Complete the verification process here:

http://secure-reward-portal.com/claim-now

Best regards,
Customer Rewards Team""",
            """From: support@paypal-service.com
To: alexander.brown@company.com
Subject: Confirm Your PayPal Account

Dear Customer,

We noticed a security issue on your account. Click below to confirm:

https://paypal-service-verify.com/security-check

Best,
PayPal Support""",
            """From: booking@airlinesupport.net
To: hannah.williams@company.com
Subject: Your Flight Reservation Is On Hold

Dear Hannah,

We need additional details to process your booking. Verify here:

http://secure-airline-booking.com/confirm-details

Kind regards,
Airline Support""",
            """From: alert@bank-secure.com
To: emily.taylor@company.com
Subject: Fraudulent Activity Alert

Dear Emily,

Unusual transactions detected. Verify your account details here:

http://bank-secure-check.com/login

Sincerely,
Bank Security Team""",
            """From: lottery@national-lottery.net
To: david.johnson@company.com
Subject: Claim Your Winning Prize of $1,000,000!

Dear Winner,

Claim your $1,000,000 by filling out your details here:

http://lottery-prize.net/claim

Congratulations,
National Lottery Team""",
            """From: verify@apple-security.com
To: sarah.wilson@company.com
Subject: Action Required: Account Suspended

Your Apple ID has been locked. Verify immediately:

https://apple-security-check.com/login

Apple Support""",
            """From: upgrade@software-update.net
To: thomas.moore@company.com
Subject: Important Update Available

Dear Thomas,

Your software is outdated. Download the patch here:

http://update-software-now.com/patch

Thank you,
Support Team""",
            """From: invoice@supplier-payments.net
To: charles.miller@company.com
Subject: Outstanding Invoice Payment Required

Your payment is overdue. Avoid penalties by settling here:

http://supplier-invoice.net/payment

Finance Department"""
        ]

        legit_examples = [
            """From: jennifer.smith@company.com
To: team@company.com
Subject: Quarterly Review Meeting - Thursday, 2 PM

Dear Team,

Reminder: Quarterly Review Meeting this Thursday at 2 PM.

Agenda:
1. Q2 Performance
2. Q3 Goals

Join via:
https://company.zoom.us/j/123456789

Best regards,
Jennifer Smith""",
            """From: mark.jones@company.com
To: dev-team@company.com
Subject: Updated Project Timeline

Hi Team,

Updated timeline attached. Review before our sync.

Thanks,
Mark Jones""",
            """From: hr@company.com
To: all-staff@company.com
Subject: Annual Benefits Enrollment Reminder

Dear Team,

Annual Benefits Enrollment open until Nov 30th. Review here:

https://company.hr-portal.com/login

Best regards,
HR Team""",
            """From: emily.davis@company.com
To: finance-team@company.com
Subject: Monthly Budget Report

Hi Finance Team,

Attached: Budget Report for October. Please review by Friday.

Thanks,
Emily Davis""",
            """From: operations@company.com
To: logistics-team@company.com
Subject: Updated Shipping Policy

Dear Team,

Policy update document attached. Review and acknowledge.

Regards,
Operations Team""",
            """From: customer.service@company.com
To: john.doe@company.com
Subject: Order Confirmation - Thank You for Shopping

Hi John,

Your order has been processed. Details attached.

Best,
Customer Service""",
            """From: training@company.com
To: staff@company.com
Subject: Mandatory Safety Training Session

Dear Team,

Mandatory training scheduled on Monday, 9 AM.

Details:
https://company.training-platform.com/safety-session

HR Department""",
            """From: manager@company.com
To: marketing-team@company.com
Subject: Campaign Launch Review

Hi Team,

Please finalize materials for Friday’s campaign review.

Thanks,
Marketing Manager""",
            """From: jane.smith@company.com
To: support@company.com
Subject: IT Assistance Request

Hi IT Support,

Experiencing login issues. Assistance needed ASAP.

Best,
Jane Smith""",
            """From: logistics@company.com
To: all-staff@company.com
Subject: New Office Supply Request Form

Dear Staff,

Submit office supply requests via our new portal:

https://company.portal.com/forms/supplies

Logistics Team"""
        ]

        # Chọn ngẫu nhiên số lượng ví dụ theo tham số nshot
        selected_phishing = random.sample(phishing_examples, self.nshot // 2)
        selected_legit = random.sample(legit_examples, self.nshot - len(selected_phishing))

        # Trả về list các ví dụ với format phù hợp
        return (
            [{"role": "user", "email": ex, "label": "phishing"} for ex in selected_phishing] +
            [{"role": "user", "email": ex, "label": "legit"} for ex in selected_legit]
        )

    def _load_rag_dataset(self):
        """
        Đọc và tạo embedding cho dataset RAG từ các file text

        Returns:
            List các document với nội dung và embedding tương ứng
        """
        examples = []
        # Đọc tất cả file .txt trong thư mục RAG
        for file_path in self.rag_dataset_path.rglob("*.txt"):
            try:
                # Đọc nội dung và tạo embedding
                content = file_path.read_text(encoding="utf-8").strip()
                embedding = self.embedding_model.encode(content)
                examples.append({"content": content, "embedding": embedding})
            except Exception as e:
                print(f"Error reading {file_path}: {e}")
        return examples

    def _organize_rag_dataset(self):
        """
        Tổ chức dataset RAG thành các cụm sử dụng K-means clustering
        để tăng tốc độ tìm kiếm context tương tự
        """
        # Lấy tất cả embedding từ dataset
        embeddings = np.array([ex["embedding"] for ex in self.rag_dataset])

        # Tính số lượng cụm phù hợp
        n_clusters = min(len(embeddings) // 10, self.MAX_CLUSTERS)

        # Thực hiện clustering
        self.kmeans = MiniBatchKMeans(n_clusters=n_clusters, random_state=42)
        clusters = self.kmeans.fit_predict(embeddings)

        # Gán cluster cho từng document
        for example, cluster_id in zip(self.rag_dataset, clusters):
            example["cluster"] = cluster_id

        # Lưu tâm các cụm để tìm kiếm nhanh
        self.cluster_centroids = self.kmeans.cluster_centers_

    def _retrieve_relevant_context(self, query_email):
        """
        Tìm các context có liên quan nhất từ dataset RAG

        Args:
            query_email: Nội dung email cần phân tích

        Returns:
            List các context tương tự nhất với trọng số tương ứng
        """
        if not self.rag_dataset or self.kmeans is None:
            return []

        # Tạo embedding cho query
        query_embedding = self.embedding_model.encode(query_email)

        # Tìm cụm gần nhất
        cluster_similarities = cosine_similarity([query_embedding], self.cluster_centroids)[0]
        closest_cluster = cluster_similarities.argmax()

        # Lấy các document trong cụm gần nhất
        cluster_examples = [ex for ex in self.rag_dataset if ex["cluster"] == closest_cluster]

        # Tính độ tương tự với từng document
        similarities = cosine_similarity([query_embedding], [ex["embedding"] for ex in cluster_examples])[0]

        # Lấy top_k document tương tự nhất
        sorted_indices = np.argsort(similarities)[::-1][:self.top_k_rag]
        return [{"content": cluster_examples[i]["content"], "weight": similarities[i]} for i in sorted_indices]


    def analyze_email(self, email_content: str) -> SimplePhishingResult:
        """
        Analyze an email to determine if it is phishing or legit.

        Args:
            email_content: The email content to analyze.

        Returns:
            SimplePhishingResult object with the analysis result.
        """
        # Chuẩn bị prompt cho LLM
        messages = [{"role": "system", "content": "You are a assistant to analyzing emails for potential phishing. Respond with ONLY a JSON object containing is_potential_phishing field: {\"is_potential_phishing\": true/false}. If you cannot determine, return {\"is_potential_phishing\": true}. Do not include any additional text or explanation."}]

        if self.few_shot_examples:
            for ex in self.few_shot_examples:
                messages.append({
                    "role": "user",
                    "content": (
                        f"Example Email:\n{ex['email']}\n"
                        f"Result: {ex['label']}"
                    )
                })

        if self.use_rag:
            for context in self._retrieve_relevant_context(email_content):
                messages.append({
                    "role": "user",
                    "content": (
                        f"Relevant Context:\n{context['content']}\n"
                        f"Similarity Score: {context['weight']:.2f}"
                    )
                })


        # Thêm email cần phân tích
        messages.append( {
                "role": "user",
                "content": (
                    f"Analyze this email:\n{email_content}\n"
                    "### Response Format:\n"
                    "Using provide information to respond with ONLY a JSON using object containing is_potential_phishing field."
                )
            })


        try:
            # Make the API call using instructor with response_model
            result = self.client.chat.completions.create(
                model=self.ollama_model_name,
                messages=messages,
                temperature=0.3,
                response_model=SimplePhishingResult
            )
            return result

        except Exception as e:
            # print(f"Error during API call: {e}")
            return SimplePhishingResult(is_potential_phishing=False)

In [ ]:
rag_dataset_path = "rag_dataset"
fine_tune=True
detector = PhishingDetector(
            rag_dataset_path=str(rag_dataset_path) if rag_dataset_path else None,
            use_rag=True,
            use_few_shot=True,
            ollama_model_name=("hf.co/daophuongkhanh201/gemma-2b:Q4_K_M" if fine_tune else "gemma:2b"))

from IPython.display import display, HTML
import ipywidgets as widgets

# GUI components
email_input = widgets.Textarea(
    value='',
    placeholder='Paste the email content here...',
    description='Email:',
    disabled=False,
    layout={'width': '100%', 'height': '200px'}
)

analyze_button = widgets.Button(
    description='Analyze Email',
    disabled=False,
    button_style='primary',
    tooltip='Click to analyze the email',
    icon='check'
)

output = widgets.Output()

def on_button_clicked(b):
    with output:
        output.clear_output(wait=False)  # Clear output immediately without waiting
        print("Analyzing email...")  # Show loading message before result
        try:
            # Call your email analysis method here
            analysis = detector.analyze_email(email_input.value)
            # Display formatted analysis result
            display(HTML(format_analysis(analysis)))
        except Exception as e:
            # Catch any errors and display them in the output
            print(f"An error occurred: {str(e)}")

# Bind the button click event to the function
analyze_button.on_click(on_button_clicked)

def format_analysis(analysis: SimplePhishingResult) -> str:
    # Format the analysis result based on phishing status
    color = "red" if analysis.is_potential_phishing else "green"
    result = f"""
    <h2 style="color: {color};">{'Potential Phishing Detected' if analysis.is_potential_phishing else 'Likely Legitimate Email'}</h2>
    """
    return result

# Display the GUI components
display(email_input, analyze_button, output)




Textarea(value='', description='Email:', layout=Layout(height='200px', width='100%'), placeholder='Paste the e…

Button(button_style='primary', description='Analyze Email', icon='check', style=ButtonStyle(), tooltip='Click …

Output()

In [ ]:
""" Legit email"""
"""
From: jennifer.smith@company.com
To: team@company.com
Subject: Quarterly Review Meeting - Thursday, 2 PM

Dear Team,

I hope this email finds you well. I'm writing to remind everyone about our upcoming Quarterly Review Meeting scheduled for this Thursday at 2 PM in the main conference room.

Agenda:
1. Q2 Performance Overview
2. Project Updates
3. Q3 Goals and Strategies
4. Open Discussion

Please come prepared with your team's updates and any questions you may have. If you're unable to attend in person, you can join via our usual video conferencing link:
https://company.zoom.us/j/123456789

Don't forget to bring your laptops for the interactive portion of the meeting.

If you have any questions or need to discuss anything before the meeting, feel free to drop by my office or send me an email.

Looking forward to seeing everyone on Thursday!

Best regards,
Jennifer Smith
Head of Operations
Company Inc.
"""

"""
From: john.doe@creativeagency.com
To: allstaff@creativeagency.com
Subject: Creative Team Collaboration - Project Kickoff

Hello Everyone,

I hope you are all doing well. I wanted to let you know that our next big project will kick off next Monday at 10 AM. We’ll be meeting in the conference room to discuss ideas, timelines, and team responsibilities.

Agenda:
1. Overview of the project brief
2. Assigning roles and responsibilities
3. Initial brainstorming session
4. Set deadlines for deliverables

Please be ready with some creative ideas that we can start building on. If you can’t make it to the meeting, we will send out the recording, but do try to attend so we can all align from the start.

Looking forward to a productive session with everyone.

Best,
John Doe
Creative Director
Creative Agency
"""

"""
From: sarah.lane@softwarecorp.com
To: devteam@softwarecorp.com
Subject: Sprint Review and Demo - Wednesday at 3 PM

Hi Team,

Just a quick reminder about our Sprint Review and Demo session happening this Wednesday at 3 PM in the conference room. We’ll go over the features developed in the last sprint and preview the upcoming tasks for the next sprint.

Agenda:
1. Sprint progress overview
2. Demonstration of new features
3. Discussion of upcoming tasks and priorities
4. Feedback and Q&A

It’s important to have your demos ready and make sure everything is up to date. If you need help preparing, don’t hesitate to reach out.

Thanks,
Sarah Lane
Product Manager
SoftwareCorp

"""

"""
From: michael.roberts@marketingfirm.com
To: marketingteam@marketingfirm.com
Subject: Upcoming Campaign Planning - Friday at 11 AM

Hello Team,

I hope you’re all having a great week! Our meeting this Friday at 11 AM will focus on planning the next big campaign. We’ll discuss the campaign’s objectives, target audience, and the creative direction.

Agenda:
1. Review of campaign goals
2. Budget and resource allocation
3. Creative direction and concept approval
4. Next steps and deadlines

Please come prepared with your thoughts and ideas. Let me know if you need anything in advance.

Best regards,
Michael Roberts
Campaign Lead
Marketing Firm

"""


'\nFrom: michael.roberts@marketingfirm.com\nTo: marketingteam@marketingfirm.com\nSubject: Upcoming Campaign Planning - Friday at 11 AM\n\nHello Team,\n\nI hope you’re all having a great week! Our meeting this Friday at 11 AM will focus on planning the next big campaign. We’ll discuss the campaign’s objectives, target audience, and the creative direction.\n\nAgenda:\n1. Review of campaign goals\n2. Budget and resource allocation\n3. Creative direction and concept approval\n4. Next steps and deadlines\n\nPlease come prepared with your thoughts and ideas. Let me know if you need anything in advance.\n\nBest regards,\nMichael Roberts\nCampaign Lead\nMarketing Firm\n\n'

In [ ]:
""" Phishing email"""
"""
From: security@amazonsupport.net
To: john.doe@company.com
Subject: Urgent: Your Amazon Account Has Been Locked

Dear Valued Customer,

We have detected unusual activity on your Amazon account. To prevent unauthorized access, we have temporarily locked your account for your protection.

To unlock your account and restore full access, please click on the link below and verify your information:

https://amaz0n-account-verify.com/unlock-account

If you do not verify your account within 24 hours, it will be permanently disabled.

Thank you for your prompt attention to this matter.

Best regards,
Amazon Security Team

This email was sent by Amazon.com. To ensure delivery to your inbox, please add security@amazonsupport.net to your address book.
"""

"""
From: support@apple.com
To: sarah.martin@company.com
Subject: Action Required: Verify Your Apple ID for Security

Dear Sarah,

We are reaching out to you because we’ve detected some unusual login attempts on your Apple ID from an unfamiliar device. For your security, we’ve temporarily locked your account to prevent unauthorized access.

To verify your identity and regain access, please follow the link below to complete the security check:

https://secure.apple.com/account/verify

This is a necessary step to ensure your Apple ID remains safe. Please complete the process within 24 hours to avoid any interruptions.

If you have any questions or need further assistance, please do not hesitate to reach out to our support team.

Thank you for your prompt attention to this matter.

Sincerely,
Apple Support Team
Apple Inc.

"""

"""
From: noreply@paypalsecurity.com
To: james.carter@company.com
Subject: PayPal Account Activity Alert

Dear James,

We are sending you this message because of a security alert on your PayPal account. Our system has detected multiple login attempts from an unrecognized device, and as a precaution, your account has been temporarily restricted.

To secure your account and restore full access, please click the link below to verify your identity:

https://www.paypal.com/secure-verification

This is important to prevent further unauthorized access. Please take action as soon as possible. If we don’t hear from you within 48 hours, your account may be permanently restricted.

Best regards,
PayPal Security Team
PayPal



"""

"""
From: security@chase.com
To: emily.wilson@company.com
Subject: Immediate Verification Required - Chase Bank Account

Dear Emily,

We’ve noticed a recent attempt to log in to your Chase account from an unfamiliar location. As a precaution, we’ve temporarily suspended your account and require verification to restore access.

To verify your information and unlock your account, please click on the link below:

https://www.chase.com/verify-account

Please complete this verification within the next 48 hours to avoid any disruptions. Your account will remain suspended until we can confirm your identity.

If you have any concerns, feel free to reach out to our customer support team.

Thank you for your cooperation.

Sincerely,
Chase Security Team
Chase Bank


"""

'\nFrom: security@chase.com\nTo: emily.wilson@company.com\nSubject: Immediate Verification Required - Chase Bank Account\n\nDear Emily,\n\nWe’ve noticed a recent attempt to log in to your Chase account from an unfamiliar location. As a precaution, we’ve temporarily suspended your account and require verification to restore access.\n\nTo verify your information and unlock your account, please click on the link below:\n\nhttps://www.chase.com/verify-account\n\nPlease complete this verification within the next 48 hours to avoid any disruptions. Your account will remain suspended until we can confirm your identity.\n\nIf you have any concerns, feel free to reach out to our customer support team.\n\nThank you for your cooperation.\n\nSincerely,\nChase Security Team\nChase Bank\n\n\n'